In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 開始使用文字嵌入 + Vertex AI 向量搜尋


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/doggy8088/generative-ai/blob/main/embeddings/intro-textemb-vectorsearch.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 執行 Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/doggy8088/generative-ai/blob/main/embeddings/intro-textemb-vectorsearch.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在 GitHub 上檢視
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/main/embeddings/intro-textemb-vectorsearch.zh.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在 Vertex AI Workbench 中開啟
    </a>
  </td>
</table>


| | |
|-|-|
|作者 | [Smitha Venkat](https://github.com/smitha-google), [Kaz Sato](https://github.com/kazunori279)|


## 簡介

在本教學課程中，你將學習如何使用 Google Cloud AI 工具，將大型語言模型的效能快速帶入企業系統。

本教學課程涵蓋以下內容 -

*   嵌入是什麼 - 它們能協助解決什麼業務挑戰？
*   使用 Vertex AI 文字嵌入了解文字
*   透過 Vertex AI 向量搜尋快速找出嵌入
*   以向量搜尋為 LLM 輸出提供根據

本教學課程根據 [部落格文章](https://cloud.google.com/blog/products/ai-machine-learning/how-to-use-grounding-for-your-llms-with-text-embeddings) 加上範例程式碼編寫。


### 前提條件

本教學課程專為具備 Python 程式設計和機器學習基本知識和經驗的開發人員而設計。

如果你並不是在 Qwiklab 中閱讀本教學課程，則你需要一個與帳單帳戶連結的 Google Cloud 專案才能執行本教學課程。請參閱 [這份文件](https://cloud.google.com/vertex-ai/docs/start/cloud-environment) 建立專案並為其設定帳單帳戶。

### 選擇執行時期環境

本筆記本可以在 Google Colab 或 [Vertex AI 工作台](https://cloud.google.com/vertex-ai-workbench) 上執行。

- 使用 Colab：按一下 [此連結](https://colab.research.google.com/github/doggy8088/generative-ai/blob/main/embeddings/intro-textemb-vectorsearch.zh.ipynb) 在 Colab 中開啟本教學課程。

- 使用工作台：如果你是第一次在你的 Google Cloud 專案中使用工作台，請開啓 [工作台控制台](https://console.cloud.google.com/vertex-ai/workbench) 並按一下「啟用」按鈕以啟用筆記本 API。然後按一下 [此連結](https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/main/embeddings/intro-textemb-vectorsearch.zh.ipynb)，然後選擇現有的筆記本或建立新的筆記本。


### 這需要多少費用？

如果你使用自己的 Cloud 專案，而非 Qwiklab 上的臨時專案，你需要支付大約幾美元來完成此教學課程。

我們將在此教學課程中使用的 Cloud 服務定價資訊如下所示：

- [Vertex AI 文字嵌入](https://cloud.google.com/vertex-ai/pricing#generative_ai_models)
- [Vertex AI 向量搜尋](https://cloud.google.com/vertex-ai/pricing#matchingengine)
- [BigQuery](https://cloud.google.com/bigquery/pricing)
- [Cloud Storage](https://cloud.google.com/storage/pricing)
- [Vertex AI Workbench](https://cloud.google.com/vertex-ai/pricing#notebooks)(如果你使用此服務) 

你可以使用 [定價計算器](https://cloud.google.com/products/calculator)根據預計使用量產生成本估計。以下是使用計算器估算成本的範例，假設你將執行這項教學課程兩次。

<img src="https://storage.googleapis.com/github-repo/img/embeddings/vs-quickstart/pricing.png" width="50%"/>

### **警告：在教學課程結束後刪除你的物件** 

如果你使用自己的 Cloud 專案，請確定在結束此教學課程後刪除所有索引、索引端點和 Cloud Storage 儲存空間 (以及你使用的 Workbench 實體)。否則，剩餘的資產會產生意外成本。


# 將生成式 AI 和 LLM 帶入生產服務

目前許多人開始思考如何將生成式 AI 和 LLM 帶入生產服務，並面臨一些挑戰。

- 「如何將 LLM 或 AI 聊天機器人與現有的 IT 系統、資料庫和業務數據整合？」
- 「我們有成千上萬的產品，如何讓 LLM 精確記住所有產品？」
- 「如何處理 AI 聊天機器人中的幻覺問題，以建立可靠的服務？」

以下是快速的解決方案：使用**嵌入** 和**向量搜尋** 進行**聯繫** 。

什麼是聯繫？什麼是嵌入和向量搜尋？在本教學中，我們將學習這些關鍵概念，以建構可靠的企業用生成式 AI 服務。但在深入探討之前，讓我們試試以下的示範。


![](https://storage.googleapis.com/gweb-cloudblog-publish/original_images/1._demo_animation.gif)

**練習：試用 Stack Overflow 語意搜尋示範：** 

這個示範作為一個 [公開即時示範](https://ai-demos.dev/) 提供。選擇「STACKOVERFLOW」並輸入任何編碼問題作為查詢，讓它針對 [Stack Overflow](https://stackoverflow.com/) 上已發布的 **800 萬** 個問題執行文字搜尋。使用某些查詢，例如「如何在 SQL 中洗牌列？」或任意程式設計問題，來嘗試文字語意搜尋。

在此教學課程中，我們將會了解如何建立一個類似的搜尋體驗 - 使用 Vertex AI 嵌入 API 和向量搜尋建立這個類型的解決方案所涉及的內容。


# 什麼是 Embedding？

隨著 LLM 的興起，為什麼 IT 工程師和 ITDM 了解它們如何運作變得重要？

在傳統的 IT 系統中，大多數資料都以結構或表格資料的方式組織，使用資料庫和搜尋引擎中的簡單關鍵字、標籤和類別。

![](https://storage.googleapis.com/github-repo/img/embeddings/textemb-vs-notebook/1.png)

相比之下，AI 驅動的服務將資料排列成一種稱為「Embedding」的簡單資料結構。

![](https://storage.googleapis.com/github-repo/img/embeddings/textemb-vs-notebook/2.png)


經過針對特定內容 (如文本、影像或任何內容) 訓練後，AI 可創造名為「嵌入空間」的空間，基本上這是內容意義的地圖。

![](https://storage.googleapis.com/github-repo/img/embeddings/textemb-vs-notebook/3.png)

AI 可以識別各內容在地圖上的位置，這就是嵌入。

![](https://storage.googleapis.com/github-repo/img/embeddings/textemb-vs-notebook/4.png)

舉一個例子：假設一篇文本討論電影、音樂和演員，其分布分別為 10%、2% 和 30%。在此情況下，AI 可以創造三維空間中的三組值：0.1、0.02 和 0.3 來進行嵌入。

![](https://storage.googleapis.com/github-repo/img/embeddings/textemb-vs-notebook/5.png)

AI 可以將具有類似意義的內容緊密放在一起。


這是 Google 如何透過各種服務 (如 Google 搜尋、YouTube、Play 和許多其他服務) 組織資料，以提供包含相關內容的搜尋結果和推薦。

Embeddings 也可應用於表示企業中的不同類別事物，例如產品、使用者、使用者活動、對話、音樂和影片、物聯網感測器信號等。


人工智慧與嵌入式技術成為人與電腦互動新方式的關鍵。

![](https://storage.googleapis.com/github-repo/img/embeddings/textemb-vs-notebook/6.png)

人工智慧將資料組織成嵌入式技術，表示使用者正在尋找的內容、內容含義，或你在企業中遇到的眾多事項。這種技術創造出新層級的使用者體驗，並正成為新標準。

如需進一步瞭解嵌入式技術，請參閱 [基礎課程：Google Machine Learning 深入淺出課程中的嵌入式技術](https://developers.google.com/machine-learning/crash-course/embeddings/video-lecture) 和 [認識人工智慧的多功能工具：Dale Markowitz 的向量嵌入式技術](https://cloud.google.com/blog/topics/developers-practitioners/meet-ais-multitool-vector-embeddings)，這些都是極佳的學習素材。


# Vertex AI 文字嵌入

使用 [Vertex AI 文字嵌入](https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings)，你可以輕鬆使用 LLM 建立文字嵌入。此產品也於 [Vertex AI 模組花園](https://cloud.google.com/model-garden) 推出

![](https://storage.googleapis.com/github-repo/img/embeddings/textemb-vs-notebook/7.png)

此 API 是為從文字中提取嵌入而設計。它可以擷取多達 3,072 個輸入 Token 的文字輸入，並輸出 768 維度的文字嵌入。


## LLM 文本嵌入商用案例

透過嵌入 API，你可以運用嵌入的創新，結合 LLM 能力，到各式各樣的文本處理任務，例如：

**LLM 驅動語義搜尋** : 文本嵌入可以用來表示使用者查詢與嵌入空間中的文件之意涵和意圖。其意涵類似於使用者查詢意圖的文件，將能透過向量的搜尋技術快速找到。模型能夠產生文本嵌入，以掌握文件中的每個句子和段落的細微差異。

**LLM 驅動文本分類** : 可在沒有任何訓練或微調 (所謂的零次學習) 之下，使用 LLM 文本嵌入進行文本分類，深入瞭解不同的脈絡。在沒有特定於任務的訓練之下，這對於過去的語言模型而言是不可能的。

**LLM 驅動推薦** : 文本嵌入可以用於推薦系統，作為訓練推薦模型的強大特徵，例如雙塔模型。模型學習查詢和候選嵌入之間的關係，藉此產生以語義產品推薦為主的次世代使用者體驗。

具備 LLM 等級深入語義理解功能，也能處理 LLM 驅動的群集、異常偵測、情緒分析等功能。


## 在「圖書館員等級」準確度下，對 800 萬筆文字進行分類

Vertex AI Text 嵌入有 768 維度的嵌入空間。正如前面所述，這些空間代表了世界上各種文字的一個龐大地圖，並按其含義分類組織。模型可根據每筆輸入文字，在此地圖中找到一個位置 (嵌入)。

透過視覺化嵌入空間，你實際上可以觀察到模型如何以「圖書館員等級」準確度來對文字進行分類。

**練習：試用 Nomic AI Atlas** 

[Nomic AI](http://nomic.ai/) 提供一個名為 Atlas 的平台，用於儲存、視覺化和與大規模且在順暢的 UI 中，與嵌入空間進行互動，他們還與 Google 合作，以視覺化 800 萬個 Stack Overflow 問題的嵌入空間。透過 Nomic AI 的協助，你可以在瀏覽器上探索此空間，放大縮小每個資料點。

嵌入空間代表了一個巨大的文字地圖，按其含義分類組織
透過每筆輸入的文字，模型可以在地圖中找到一個位置 (嵌入) 
就像一位圖書館員閱讀數以百萬計的文字，使用數以百萬計的納米分類將其分類

在此處 [嘗試探索](https://atlas.nomic.ai/map/edaff028-12b5-42a0-8e8b-6430c9b8222b/bcb42818-3581-4fb5-ac30-9883d01f98ec)。縮小到幾個類別，點選每個點，看看 LLM 如何在空間中對相似的問題進行緊密分類。

![](https://storage.googleapis.com/gweb-cloudblog-publish/images/4._Nomic_AI_Atlas.max-2200x2200.png)

### 圖書館員等級的語意理解

以下是用於 Stack Overflow 問題的嵌入 API 進行的圖書館員等級語意理解的範例。

![](https://storage.googleapis.com/gweb-cloudblog-publish/images/5._semantic_understanding.max-2200x2200.png)

例如，模型認為「將請求行移至標頭框架是否需要進行應用程式變更？」這個問題類似於「在 HTTP1x 上開發的應用程式是否需要修改才能在 HTTP2 上執行？」這個問題。這是因為模型知道這兩個問題都在討論支援 HTTP2 標頭框架所需的變更為何。

請注意，此示範並不需要使用特定於電腦程式的資料集進行任何訓練或微調。這是 LLM 零次學習功能的創新部分。它可應用於各行各業，包括金融、醫療保健、零售、製造、營建、媒體等，用於針對特定行業的商業文件執行深入的語意搜尋，而無需花費時間和成本去收集特定產業的資料集與訓練模型。


# 文字嵌入實作

讓我們用實際範例程式碼嘗試使用文字嵌入。


## 設定

在開始使用 Vertex AI 服務之前，我們需要設定下列事項。

* 安裝 Python SDK
* 環境變數
* 驗證 (僅限 Colab) 
* 啟用 API
* 設定 IAM 權限


### 安裝 Python SDK

Vertex AI、Cloud Storage 和 BigQuery API 都能透過多種方式存取，包含 REST API 和 Python SDK。在本教學中，我們將使用 SDK。


In [ ]:
!pip install --upgrade --user google-cloud-aiplatform google-cloud-storage 'google-cloud-bigquery[pandas]'

### 重新啟動目前的執行階段

要在此 Jupyter 執行階段中使用新安裝的套件，你必須重新啟動執行階段。你可以執行下列Cell來執行此項操作，如此將重新啟動目前的Kernel。


In [2]:
# Restart kernel after installs so that your environment can access the new packages
import IPython
import time

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ Kernel將重新啟動。請等待它完成，再繼續執行下一個步驟。⚠️</b>
</div>


### 環境變數

設定環境變數。若有此詢問，請將以下 `[your-project-id]` 替換成你的專案 ID 並執行。


In [ ]:
# get project ID
PROJECT_ID = ! gcloud config get project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1"
if PROJECT_ID == "(unset)":
    print(f"Please set the project ID manually below")

In [ ]:
# define project information
if PROJECT_ID == "(unset)":
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# generate an unique id for this session
from datetime import datetime

UID = datetime.now().strftime("%m%d%H%M")

### 驗證 (僅限 Colab) 

如果你是在 Colab 上執行這個筆記本，你需要執行以下單元格驗證。如果你使用 Vertex AI Workbench 則不需要執行此步驟，因為它已預先進行驗證。


In [ ]:
import sys

# if it's Colab runtime, authenticate the user with Google Cloud
if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### 啟用 API

執行以下動作，以針對執行此 Google Cloud 專案的 Compute Engine、Vertex AI、Cloud 儲存空間和 BigQuery 啟用 API。


In [ ]:
! gcloud services enable compute.googleapis.com aiplatform.googleapis.com storage.googleapis.com bigquery.googleapis.com --project {PROJECT_ID}

### 設定 IAM 權限

此外，我們需要為使用那些服務的預設服務帳戶新增存取權限。

- 移至主控台中的 [IAM 頁面](https://console.cloud.google.com/iam-admin/)
- 找出預設計算服務帳戶的主體。它應類似於：`<project-number>-compute@developer.gserviceaccount.com`
- 按一下右方的編輯按鈕，然後按一下 `新增另一個角色`，以將 `Vertex AI 使用者`、`BigQuery 使用者` 和 `儲存空間管理員` 新增到該帳戶。

這將看起來像這樣：

![](https://storage.googleapis.com/github-repo/img/embeddings/vs-quickstart/iam-setting.png)


## 開始使用 Vertex AI 文字嵌入

現在可以開始使用嵌入式！


### 數據準備

我們將使用 [Stack Overflow 公共數據集](https://console.cloud.google.com/marketplace/product/stack-exchange/stack-overflow) 託管在 BigQuery 表 `bigquery-public-data.stackoverflow.posts_questions` 中。這個是包含 2300 萬列的大型數據集，無法放進記憶體中。我們將這個教學限制為 1000 列。


In [ ]:
# load the BQ Table into a Pandas Dataframe
import pandas as pd
from google.cloud import bigquery

QUESTIONS_SIZE = 1000

bq_client = bigquery.Client(project=PROJECT_ID)
QUERY_TEMPLATE = """
        SELECT distinct q.id, q.title
        FROM (SELECT * FROM `bigquery-public-data.stackoverflow.posts_questions`
        where Score > 0 ORDER BY View_Count desc) AS q
        LIMIT {limit} ;
        """
query = QUERY_TEMPLATE.format(limit=QUESTIONS_SIZE)
query_job = bq_client.query(query)
rows = query_job.result()
df = rows.to_dataframe()

# examine the data
df.head()

### 呼叫 API 以產生內嵌

使用 Stack Overflow 資料集，我們將使用 `標題` 欄 (問題標題)，並利用 Embeddings for Text API 為它生成內嵌。在 SDK 的 [vertexai](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai) 套件中提供了該 API。

你可能會看到 TensorFlow 函式庫的一些警告訊息，但你可以忽略它們。


In [ ]:
# init the vertexai package
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

從套件匯入 [TextEmbeddingModel](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.language_models.TextEmbeddingModel) 並獲取模型。


In [ ]:
# Load the text embeddings model
from vertexai.preview.language_models import TextEmbeddingModel

model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")

在本教學中，我們將使用 `textembedding-gecko@001` 模型來取得文字組態。請查看文件 [支援的模型](https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings#supported_models) 中的支援模型清單。

取得模型後，你可以呼叫它的 [get_embeddings](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.language_models.TextEmbeddingModel#vertexai_language_models_TextEmbeddingModel_get_embeddings) 函式來取得組態。你每次呼叫可以傳遞最多 5 段文字。但有一個但書。預設上，文字組態 API 會將「每分鐘請求次數」配額設定為新雲端專案 60 次，使用歷程中的專案則是 600 次 (請參閱 [配額和限制](https://cloud.google.com/vertex-ai/docs/quotas#request_quotas) 來確認 `base_model:textembedding-gecko` 的最新配額值)。因此，與其直接使用該函式，你可能希望定義一個包裝器 (如下所示)，將每秒呼叫次數限制在 10 次以下，並每次傳遞 5 段文字。


In [ ]:
import time
import tqdm  # to show a progress bar

# get embeddings for a list of texts
BATCH_SIZE = 5


def get_embeddings_wrapper(texts):
    embs = []
    for i in tqdm.tqdm(range(0, len(texts), BATCH_SIZE)):
        time.sleep(1)  # to avoid the quota error
        result = model.get_embeddings(texts[i : i + BATCH_SIZE])
        embs = embs + [e.values for e in result]
    return embs

以下程式碼將為問題標題取得內嵌，並將其新增為 DataFrame 的新欄位 `embedding`。這將會花費幾分鐘。


In [ ]:
# get embeddings for the question titles and add them as "embedding" column
df = df.assign(embedding=get_embeddings_wrapper(list(df.title)))
df.head()

## 查看嵌入的相似性

讓我們藉由快速計算它們之間的相似性並排序，看看這些嵌入是如何根據其含義組織在嵌入空間中的。

由於嵌入是向量，因此你可以使用其中一種流行的指標 (例如以下指標) 來計算兩個嵌入之間的相似性：

![](https://storage.googleapis.com/github-repo/img/embeddings/textemb-vs-notebook/8.png)

我們應該使用哪個指標？通常取決於每一個模型是如何訓練的。對於模型 `textembedding-gecko@001`，我們需要使用內積 (點積)。

在以下程式碼中，它隨機擷取一個問題，並使用 numpy 的 `np.dot` 函式來計算問題和其他問題之間的相似性。


In [ ]:
import random
import numpy as np

# pick one of them as a key question
key = random.randint(0, len(df))

# calc dot product between the key and other questions
embs = np.array(df.embedding.to_list())
similarities = np.dot(embs[key], embs.T)

# print similarities for the first 5 questions
similarities[:5]

最後，利用相似性對問題進行排序，並列印清單。


In [ ]:
# print the question
print(f"Key question: {df.title[key]}\n")

# sort and print the questions by similarities
sorted_questions = sorted(
    zip(df.title, similarities), key=lambda x: x[1], reverse=True
)[:20]
for i, (question, similarity) in enumerate(sorted_questions):
    print(f"{similarity:.4f} {question}")

# 使用 Vertex AI Vector Search 快速找到嵌入

如上所述，你可以透過計算嵌入之間的距離或相似度來找出類似的嵌入。

不過，當你有多達數百萬或數十億個嵌入時，這並不輕鬆。例如，如果你有 100 萬個擁有 768 個維度的嵌入，你就需要重複進行 100 萬乘以 768 次的距離計算。這將需要數秒的時間 – 速度太慢了。


因此，研究員一直在研究稱為 [近似最近鄰 (ANN)](https://en.wikipedia.org/wiki/Nearest_neighbor_search) 的技術，以進行更快速的搜尋。 ANN 使用「向量量化」將空間分隔成具有樹狀結構的多重空間。這類似於關聯式資料庫中的索引，用於改善查詢效能，並支援針對數十億個嵌入進行快速且可擴充的搜尋。

隨著 LLM 的崛起，ANN 將以向量搜尋技術的名稱迅速普及。

![](https://storage.googleapis.com/gweb-cloudblog-publish/images/7._ANN.1143068821171228.max-2200x2200.png)

2020 年，Google Research 發布名為 [ScaNN](https://ai.googleblog.com/2020/07/announcing-scann-efficient-vector.html) 的新 ANN 演算法。它被認為是業界最佳的 ANN 演算法之一，也是 Google Search、YouTube 及眾多其他 Google 服務中搜尋和推薦最主要的基礎。


## Vertex AI 向量搜尋是什麼？

Google Cloud 開發人員可以透過 [Vertex AI 向量搜尋](https://cloud.google.com/vertex-ai/docs/vector-search/overview)(之前稱為比對引擎) 充分利用 Google 的向量搜尋技術。有了這個全受管服務，開發人員可以將內嵌程式碼新增至索引，並使用關鍵內嵌程式碼發出搜尋查詢，以進行超快速的向量搜尋。在 Stack Overflow 範例中，向量搜尋可以在數十毫秒內從 8 百萬個內嵌程式碼中找到相關問題。

---0---

如果你追求高可擴充性、可用性和可維護性，以供生產系統使用，使用向量搜尋，你不必花費大量時間和金錢從頭建立自己的向量搜尋服務，或使用開放原始碼工具。


## 開始使用向量搜尋

當你已有嵌入時，便很容易開始使用向量搜尋。在此部分，我們將遵循下列步驟。

### 設定向量搜尋
- 將嵌入儲存在雲端儲存空間的 JSON 檔案中
- 建立索引
- 建立索引端點
- 將索引部署到端點

### 使用向量搜尋

- 透過端點查詢

### **給 Colab 使用者的提示** 

如果你使用 Colab 進行此教學課程，你可能會在後面的部分等待索引建立和部署時失去執行階段，因為這需要花費數十分鐘。在這種情況下，請再次使用新的執行個體執行以下部分，以恢復執行階段：[安裝 Python SDK、環境變數和驗證](https://colab.research.google.com/drive/1xJhLFEyPqW0qvKiERD6aYgeTHa6_U50N?resourcekey=0-2qUkxckCjt6W03AsqvZHhw#scrollTo=AtXnXhF8U-8R&line=9&uniqifier=1)。

然後，使用 [工具程式](https://colab.research.google.com/drive/1xJhLFEyPqW0qvKiERD6aYgeTHa6_U50N?resourcekey=0-2qUkxckCjt6W03AsqvZHhw#scrollTo=BE1tELsH-u8N&line=1&uniqifier=1) 回復索引和索引端點，然後繼續執行其餘部分。


### 將嵌入儲存在 JSON 檔案中

要載入嵌入至 Vector Search，我們需要以 JSONL 格式將其儲存在 JSON 檔案中。更多資訊請參閱說明文件中的 [輸入資料格式和結構](https://cloud.google.com/vertex-ai/docs/matching-engine/match-eng-setup/format-structure#data-file-formats)。

首先，以 JSONL 格式從 DataFrame 中匯出 `id` 和 `embedding` 欄，並儲存它。


In [ ]:
# save id and embedding as a json file
jsonl_string = df[["id", "embedding"]].to_json(orient="records", lines=True)
with open("questions.json", "w") as f:
    f.write(jsonl_string)

# show the first few lines of the json file
! head -n 3 questions.json

然後，建立新的 Cloud Storage 儲存空間，並將檔案複製到其中。


In [ ]:
BUCKET_URI = f"gs://{PROJECT_ID}-embvs-tutorial-{UID}"
! gsutil mb -l $LOCATION -p {PROJECT_ID} {BUCKET_URI}
! gsutil cp questions.json {BUCKET_URI}

### 建立索引

現在已準備好將嵌入資料載入到 Vector Search。其 API 可在 SDK 的[aiplatform](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform) 套件中取得。


In [ ]:
# init the aiplatform package
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION)

使用它的 `create_tree_ah_index` 函式建立 [MatchingEngineIndex](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.MatchingEngineIndex)(Matching Engine 是 Vector Search 的先前名稱)。


In [ ]:
# create index
my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=f"embvs-tutorial-index-{UID}",
    contents_delta_uri=BUCKET_URI,
    dimensions=768,
    approximate_neighbors_count=20,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
)

透過呼叫 `create_tree_ah_index` 函式，它會開始建置索引。如果資料集較小，這將會花費幾分鐘；否則，將會花費約 50 分鐘或更多，具體取決於資料集的大小。你可以在 [Vector Search Console > INDEXES 標籤](https://console.cloud.google.com/vertex-ai/matching-engine/indexes)上查看索引建立的狀態。

![](https://storage.googleapis.com/github-repo/img/embeddings/vs-quickstart/creating-index.png)

#### 建立索引的參數

- `contents_delta_uri`：你用於儲存嵌入式 JSON 檔案的「Google Cloud Storage」目錄 URI。
- `dimensions`：每個嵌入的維度大小。在本例中，我們使用文字嵌入 API 的嵌入，因此為 768。
- `approximate_neighbors_count`：我們想要在一般情況中擷取的類似項目數量
- `distance_measure_type`：用於丈量嵌入之間距離/相似性的指標。在本例中為 `DOT_PRODUCT_DISTANCE`

參閱 [文件中](https://cloud.google.com/vertex-ai/docs/vector-search/create-manage-index)的詳細資料，以進一步了解建立索引和參數。

#### 批次更新或串流更新？
索引有兩種類型：*批次更新*索引 (本教學課程使用) 和*串流更新*索引。可以透過批次處理更新批次更新索引，而串流更新索引可以在實際時間更新。後者更適合於你想要更頻繁地在索引中新增或更新每個嵌入，並且對提供最新嵌入至關重要的使用案例，例如電子商務產品搜尋。


### 建立索引端點並部署索引

若要使用索引，你需要建立一個 [索引端點](https://cloud.google.com/vertex-ai/docs/vector-search/deploy-index-public)。它會像伺服器執行個體一樣，接受索引的查詢要求。


In [ ]:
# create IndexEndpoint
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=f"embvs-tutorial-index-endpoint-{UID}",
    public_endpoint_enabled=True,
)

本教學程式使用 [公共端點](https://cloud.google.com/vertex-ai/docs/vector-search/setup/setup#choose-endpoint)，不支援 [虛擬專用雲端 (VPC)](https://cloud.google.com/vpc/docs/private-services-access)，除非你對 VPC 有特定需求，建議你使用公共端點，儘管名稱中帶有「公共」，但這並不表示其他人可透過公共網際網路存取它，它的運作方式和其他 Vertex AI 服務的端點相同，透過 IAM 預設來保護。在建立明確的 IAM 權限之前，如我們之前所述，沒有人能夠存取端點。


使用 Index 終端點，指派唯一的已部署索引 ID，以部署 Index。


In [ ]:
DEPLOYED_INDEX_ID = f"embvs_tutorial_deployed_{UID}"

In [ ]:
# deploy the Index to the Index Endpoint
my_index_endpoint.deploy_index(index=my_index, deployed_index_id=DEPLOYED_INDEX_ID)

如果這是第一次將索引部署到索引端點，自動打造並初始化該索引後端約需要 25 分鐘。第一次部署後，它就會在數秒內完成。如需查看索引部署狀態，請開啟 [向量搜尋主控台 > 索引端點索引標籤](https://console.cloud.google.com/vertex-ai/matching-engine/index-endpoints)，然後按一下索引端點。

<img src="https://storage.googleapis.com/github-repo/img/embeddings/vs-quickstart/deploying-index.png" width="70%">


### 執行查詢

最後它可以準備好使用 Vector Search。在以下程式碼中，它會建立測試問題的嵌入，並使用 Vector Search 找出類似的問題。


In [ ]:
test_embeddings = get_embeddings_wrapper(["How to read JSON with Python?"])

In [ ]:
# Test query
response = my_index_endpoint.find_neighbors(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=test_embeddings,
    num_neighbors=20,
)

# show the result
import numpy as np

for idx, neighbor in enumerate(response[0]):
    id = np.int64(neighbor.id)
    similar = df.query("id == @id", engine="python")
    print(f"{neighbor.distance:.4f} {similar.title.values[0]}")

`find_neighbors` 函式僅需幾毫秒就能取得類似項目，即使索引中有數十億項目，這要歸功於 ScaNN 演算法。向量搜尋也支援 [自動擴充](https://cloud.google.com/vertex-ai/docs/vector-search/deploy-index-public#autoscaling)，其能根據工作負載的需求自動調整節點數量。


# 重要：清理

如果你使用的是你自己的 Cloud 專案，而非 Qwiklab 上的暫存專案，請務必在完成本教學課程後刪除所有索引、索引端點和 Cloud Storage 儲存區。否則，剩餘的物件會**產生額外費用** 。

如果你使用的是 Workbench，你可能也需要從 [控制台](https://console.cloud.google.com/vertex-ai/workbench) 刪除筆記本。


In [ ]:
# wait for a confirmation
input("Press Enter to delete Index Endpoint, Index and Cloud Storage bucket:")

# delete Index Endpoint
my_index_endpoint.undeploy_all()
my_index_endpoint.delete(force=True)

# delete Index
my_index.delete()

# delete Cloud Storage bucket
! gsutil rm -r {BUCKET_URI}

# 摘要

## 使用 Vertex AI Vector Search 接地 LLM 輸出

正如我們所見，透過結合 Embeddings API 和 Vector Search，你可以使用嵌入式處理「接地」LLM 輸出至真實業務資料，並具有低延遲。

例如，如果一位使用者提出問題，Embeddings API 可以將其轉換為嵌入式，並在 Vector Search 中發出查詢以在索引中尋找類似的嵌入式。這些嵌入式表示資料庫中的實際業務資料。由於我們僅擷取業務資料，並且不會產生任何人工文字，因此結果中沒有出現幻覺的風險。

![](https://storage.googleapis.com/gweb-cloudblog-publish/original_images/10._grounding.png)

### 問題與答案之間的差異

在本教學課程中，我們使用了 Stack Overflow 資料集。之所以我們得使用它，是有原因的；由於該資料集包含許多組**問題和答案** ，因此你只要找到類似於問題的問題，就能為其找到答案。

在許多業務使用案例中，問題和答案的語意 (含義) 是不同的。另外，可能有時候你會希望在結果中加入各種推薦或個人化項目，例如電子商務網站上的產品搜尋。

在這些情況下，簡單的語意搜尋無法順利運作。這比較像是一個推薦系統問題，你可能希望訓練一個模型 (例如雙塔模式)，以了解問題嵌入式空間與答案嵌入式空間之間的關係。此外，許多製作系統會在語意搜尋後加入重新排序階段，以提升搜尋品質。歡迎參閱 [使用 TensorFlow 推薦工具和 Vertex AI 匹配引擎擴充深度擷取](https://cloud.google.com/blog/products/ai-machine-learning/scaling-deep-retrieval-tensorflow-two-towers-architecture) 以進一步了解。

### 語意搜尋 + 關鍵字搜尋的混合

你在製作系統中將會遇到的另一個常見挑戰，是支援關鍵字搜尋與語意搜尋相結合。例如，對於電子商務產品搜尋，你可能希望讓使用者能透過輸入產品名稱或型號來尋找產品。由於 LLM 沒有記住這些產品名稱或型號，因此語意搜尋無法處理這些「常見」的搜尋功能。

[Vertex AI Search](https://cloud.google.com/blog/products/ai-machine-learning/vertex-ai-search-and-conversation-is-now-generally-available)是另一個你可能會考慮用於這些需求的產品。而 Vector Search 僅提供簡單的語意搜尋功能，但 Search 提供整合的搜尋解決方案，結合語意搜尋、關鍵字搜尋、重新排序和篩選，並作為開箱即用的工具提供。

### 檢索擴充生成 (RAG) 怎麼辦？

在本教學課程中，我們探討了 LLM 嵌入式與向量搜尋的簡單組合。從這個出發點，你也可以將設計延伸至 [檢索擴充生成 (RAG)](https://www.google.com/search?q=Retrieval+Augmented+Generation+(RAG)&oq=Retrieval+Augmented+Generation+(RAG))。

RAG 是一個實作接地以 LLM 及文字聊天 UI 的熱門架構模式。這個構想是將 LLM 文字聊天 UI 作為前端，用於搭配向量搜尋的文件擷取以及結果摘要。

![](https://storage.googleapis.com/gweb-cloudblog-publish/images/Figure-7-Ask_Your_Documents_Flow.max-529x434.png)

這兩種方案之間有一些優缺點。

| | Emb + vector search | RAG |
|---|---|---|
| 設計 | 簡單 | 複雜 |
| UI | 文字搜尋 UI | 文字聊天 UI |
| 結果摘要 | 否 | 是 |
| 多輪 (考量脈絡)  | 否 | 是 |
| 延遲 | 毫秒 | 秒 |
| 成本 | 較低 | 較高 |
| 幻覺 | 沒有風險 | 有些風險 |

我們在本教學課程中探討的 Embedding + vector search 模式，提供搭配 LLM 智慧的簡單、快速且低成本語意搜尋功能。RAG 在其中加入考量脈絡的文字聊天體驗和結果摘要。雖然 RAG 提供較「生成式 AI 風格」的體驗，但它也增加了文字生成過程中幻覺和較高成本與時間的風險。

若要進一步了解如何建構 RAG 解決方案，你可以參閱 [使用 Vertex AI PaLM API 和 LangChain 輕鬆建立生成式 AI 應用程式](https://cloud.google.com/blog/products/ai-machine-learning/generative-ai-applications-with-vertex-ai-palm-2-models-and-langchain)。

## 資源

若要進一步了解，請查看以下資源：

### 文件記錄

[Vertex AI Text API 嵌入式文件說明](https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings)

[向量搜尋文件說明](https://cloud.google.com/vertex-ai/docs/matching-engine/overview)

### 向量搜尋網誌貼文

[Vertex Matching Engine：超快速且大規模可擴充的近鄰搜尋](https://cloud.google.com/blog/products/ai-machine-learning/vertex-matching-engine-blazing-fast-and-massively-scalable-nearest-neighbor-search)

[使用 Google 的向量搜尋科技，極速尋找任何事物](https://cloud.google.com/blog/topics/developers-practitioners/find-anything-blazingly-fast-googles-vector-search-technology)

[在 Vertex AI 中使用串流吸收功能實現即時 AI](https://cloud.google.com/blog/products/ai-machine-learning/real-time-ai-with-google-cloud-vertex-ai)

[Mercari 善用 Google 的向量搜尋科技來打造新的市集](https://cloud.google.com/blog/topics/developers-practitioners/mercari-leverages-googles-vector-search-technology-create-new-marketplace)

[使用 Vertex AI Matching Engine 推薦新聞文章](https://cloud.google.com/blog/products/ai-machine-learning/recommending-articles-using-vertex-ai-matching-engine)

[什麼是多模式搜尋：「具備視覺效果的 LLM」改變企業](https://cloud.google.com/blog/products/ai-machine-learning/multimodal-generative-ai-search)


# 實用工具

有時建立或部署索引需要數十分鐘，而你可能會失去與 Colab 執行階段的連線。在這種情況下，不要再次建立或部署新的索引，你可以查看 [向量搜尋控制台](https://console.cloud.google.com/vertex-ai/matching-engine/index-endpoints) 並取得現有索引以繼續。


## 取得現有的索引

若要取得現有的索引物件，請將下列的 [your-index-id] 替換成索引 ID 並執行Cell。你可以在 [Vector 搜尋控制台> 索引標籤] 中查看 ID (https://console.cloud.google.com/vertex-ai/matching-engine/indexes)。


In [ ]:
my_index_id = "[your-index-id]"  # @param {type:"string"}
my_index = aiplatform.MatchingEngineIndex(my_index_id)

## 取得現有的索引端點

若要取得已存在的「索引端點」物件，請將下列的 `[your-index-endpoint-id]` 替換為索引端點 ID，然後執行Cell。你可以在 [Vector Search Console > INDEX ENDPOINTS 索引標籤](https://console.cloud.google.com/vertex-ai/matching-engine/index-endpoints) 頁面上查看 ID。


In [ ]:
my_index_endpoint_id = "[your-index-endpoint-id]"  # @param {type:"string"}
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(my_index_endpoint_id)